# 데이터 로드, 가공, 정제

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('지하철.csv')

df.drop(['작업일자'], axis=1, inplace=True)
df['지하철역'] = df['지하철역'].str.strip()
#df['지하철역']

info = df.iloc[:,:3]
info.columns = ['년월','호선','역명']
#info.head()

# 승차
riding = df.iloc[:, 3:]
riding = riding.iloc[:,[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46]]
riding.rename(columns={'04시-05시 승차인원':'4시', '05시-06시 승차인원':'5시', '06시-07시 승차인원':'6시','07시-08시 승차인원':'7시',
                       '08시-09시 승차인원':'8시','09시-10시 승차인원':'9시','10시-11시 승차인원':'10시','11시-12시 승차인원':'11시',
                       '12시-13시 승차인원':'12시','13시-14시 승차인원':'13시','14시-15시 승차인원':'14시','15시-16시 승차인원':'15시',
                      '16시-17시 승차인원':'16시','17시-18시 승차인원':'17시','18시-19시 승차인원':'18시','19시-20시 승차인원':'19시',
                      '20시-21시 승차인원':'20시','21시-22시 승차인원':'21시','22시-23시 승차인원':'22시','23시-24시 승차인원':'23시',
                      '00시-01시 승차인원':'24시','01시-02시 승차인원':'1시','02시-03시 승차인원':'2시','03시-04시 승차인원':'3시'}, inplace=True)
riding = pd.concat([info, riding], axis=1)

# 하차
stopover = df.iloc[:, 3:]
stopover = stopover.iloc[:,[1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47]]
stopover.rename(columns={'04시-05시 하차인원':'4시', '05시-06시 하차인원':'5시', '06시-07시 하차인원':'6시','07시-08시 하차인원':'7시',
                       '08시-09시 하차인원':'8시','09시-10시 하차인원':'9시','10시-11시 하차인원':'10시','11시-12시 하차인원':'11시',
                       '12시-13시 하차인원':'12시','13시-14시 하차인원':'13시','14시-15시 하차인원':'14시','15시-16시 하차인원':'15시',
                      '16시-17시 하차인원':'16시','17시-18시 하차인원':'17시','18시-19시 하차인원':'18시','19시-20시 하차인원':'19시',
                      '20시-21시 하차인원':'20시','21시-22시 하차인원':'21시','22시-23시 하차인원':'22시','23시-24시 하차인원':'23시',
                      '00시-01시 하차인원':'24시','01시-02시 하차인원':'1시','02시-03시 하차인원':'2시','03시-04시 하차인원':'3시'}, inplace=True)
stopover = pd.concat([info, stopover], axis=1)
#stopover.head()









# 데이터 가공, 정제 (데이터 프레임 조작)

In [ ]:
import pandas as pd
import numpy as np

# 승하차 인원 합계
riding['합계'] = riding.sum(axis=1)
stopover['합계'] = stopover.sum(axis=1)
#riding.head()

# 승차
df_r = riding[['년월', '호선', '역명', '합계']]
#df_r.head()

# 하차
df_s = stopover[['년월', '호선', '역명', '합계']]
#df_s.head()

# 평균으로 일일 이용객 합계
# 승차
dfg_r = df_r.groupby(['년월','호선','역명'])['합계'].mean()
#dfg_r.head()
# 하차
dfg_s = df_s.groupby(['년월','호선','역명'])['합계'].mean()
#dfg_s.head()

# 데이터프레임 수정
# 승차
dfg2_r = pd.DataFrame(dfg_r)
dfg2_r = dfg2_r.reset_index()
#dfg2_r.head()

# 하차
dfg2_s = pd.DataFrame(dfg_s)
dfg2_s = dfg2_s.reset_index()
#dfg2_s.head()

# 합계 -> 이용객수
dfg2_r.columns = ['년월', '호선', '역명', '이용객수']
dfg2_s.columns = ['년월', '호선', '역명', '이용객수']

dfg3_r = pd.DataFrame(round(dfg2_r.groupby(['호선', '역명'])['이용객수'].mean()))
dfg3_s = pd.DataFrame(round(dfg2_s.groupby(['호선', '역명'])['이용객수'].mean()))

# 이용객수로 정렬
dfg3_r = dfg3_r.sort_values(by='이용객수', ascending=False)
dfg3_s = dfg3_s.sort_values(by='이용객수', ascending=False)
#dfg3_s.head(20)

# 인덱스 리빌드
dfg4_r = pd.DataFrame(dfg3_r)
dfg4_r = dfg4_r.reset_index()
#dfg4_r.head()

dfg4_s = pd.DataFrame(dfg3_s)
dfg4_s = dfg4_s.reset_index()
#dfg4_s.head()


# 데이터 가공, 정제 (데이터 결합)

In [3]:
import pandas as pd
import numpy as np

dinfo = pd.read_csv('map_utf.csv')

dinfo.columns = ['역명', '호선', '위도', '경도']
dinfo.head()

dinfo['역명'] = dinfo['역명'].str.strip()
dinfo['역명'] = dinfo['역명'].str.replace(" ", "")

dfg4_r['호선'].unique()

# 1~9 호선 까지만 분리.
dfg4_r = dfg4_r[dfg4_r['호선'].isin(['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선'])]
dfg4_s = dfg4_s[dfg4_s['호선'].isin(['1호선', '2호선', '3호선', '4호선', '5호선', '6호선', '7호선', '8호선', '9호선'])]

dfg4_r.loc[dfg4_r['호선']=='1호선', '호선'] = 1
dfg4_r.loc[dfg4_r['호선']=='2호선', '호선'] = 2
dfg4_r.loc[dfg4_r['호선']=='3호선', '호선'] = 3
dfg4_r.loc[dfg4_r['호선']=='4호선', '호선'] = 4
dfg4_r.loc[dfg4_r['호선']=='5호선', '호선'] = 5
dfg4_r.loc[dfg4_r['호선']=='6호선', '호선'] = 6
dfg4_r.loc[dfg4_r['호선']=='7호선', '호선'] = 7
dfg4_r.loc[dfg4_r['호선']=='8호선', '호선'] = 8
dfg4_r.loc[dfg4_r['호선']=='9호선', '호선'] = 9

dfg4_s.loc[dfg4_s['호선']=='1호선', '호선'] = 1
dfg4_s.loc[dfg4_s['호선']=='2호선', '호선'] = 2
dfg4_s.loc[dfg4_s['호선']=='3호선', '호선'] = 3
dfg4_s.loc[dfg4_s['호선']=='4호선', '호선'] = 4
dfg4_s.loc[dfg4_s['호선']=='5호선', '호선'] = 5
dfg4_s.loc[dfg4_s['호선']=='6호선', '호선'] = 6
dfg4_s.loc[dfg4_s['호선']=='7호선', '호선'] = 7
dfg4_s.loc[dfg4_s['호선']=='8호선', '호선'] = 8
dfg4_s.loc[dfg4_s['호선']=='9호선', '호선'] = 9

# 역정보와 이용객수 병합
# 승차
df_m_r = pd.merge(dfg4_r, dinfo, how='inner')
#df_m_r.head()
# 하차
df_m_s = pd.merge(dfg4_s, dinfo, how='inner')
#df_m_s

# 중복제거
df_m_r_2 = df_m_r.drop_duplicates('이용객수', keep='first')
df_m_s_2 = df_m_s.drop_duplicates('이용객수', keep='first')

# 상위, 하위 30개 역 표시
# 승차
dfm_top_r = df_m_r_2.head(30)
dfm_bottom_r = df_m_r_2.tail(30)


# 하차
dfm_top_s = df_m_s_2.head(30)
dfm_bottom_s = df_m_s_2.tail(30)



# 한글폰트 설치

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
#fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

#font = fm.FontProperties(fname=fontpath, size=9)

#fm._rebuild()



fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설치


In [5]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False

# 한글 폰트 설정
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'}) # 폰트 설치

# 데이터 시각화 (지도)

In [6]:
# 지도시각화
import folium

# 승차 상위 이용객 30개역 표시
map1 = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for item in dfm_top_r.index:
    lat = dfm_top_r.loc[item, '위도']
    long = dfm_top_r.loc[item, '경도']
    folium.CircleMarker([lat,long],
                                   radius=dfm_top_r.loc[item, '이용객수']/70000,
                                   color='green',
                                    fill = True).add_to(map1)
map1


In [7]:
# 승차 하위 이용객 30개역 표시
map2 = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for item in dfm_bottom_r.index:
    lat = dfm_bottom_r.loc[item, '위도']
    long = dfm_bottom_r.loc[item, '경도']
    folium.CircleMarker([lat,long],
                                   radius=dfm_bottom_r.loc[item, '이용객수']/20000,
                                   color='green',
                                    fill = True).add_to(map2)

map2

In [8]:
# 하차 상위 이용객 30개역 표시
map3 = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for item in dfm_top_s.index:
    lat = dfm_top_s.loc[item, '위도']
    long = dfm_top_s.loc[item, '경도']
    folium.CircleMarker([lat,long],
                                   radius=dfm_top_s.loc[item, '이용객수']/70000,
                                   color='green',
                                    fill = True).add_to(map3)

map3

In [9]:
# 하차 하위 이용객 30개역 표시
map4 = folium.Map(location=[37.5502, 126.982], zoom_start=11)

for item in dfm_bottom_s.index:
    lat = dfm_bottom_s.loc[item, '위도']
    long = dfm_bottom_s.loc[item, '경도']
    folium.CircleMarker([lat,long],
                                   radius=dfm_bottom_s.loc[item, '이용객수']/20000,
                                   color='green',
                                    fill = True).add_to(map4)

map4

# 데이터 시각화 (차트)

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 호선별 승차 이용객수 (상위30)
fig=px.bar(dfm_top_r, x='호선', y='이용객수',
color='호선',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)



In [10]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 호선별 승차 이용객수 (하위30)
fig=px.bar(dfm_bottom_r, x='호선', y='이용객수',
color='호선',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)

In [11]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 호선별 하차 이용객수 (상위30)
fig=px.bar(dfm_top_s, x='호선', y='이용객수',
color='호선',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 호선별 하차 이용객수 (하위30)
fig=px.bar(dfm_bottom_s, x='호선', y='이용객수',
color='호선',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 역별 승차 이용객수 (상위30)
fig=px.bar(dfm_top_r, x='역명', y='이용객수',
color='역명',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 역별 승차 이용객수 (하위30)
fig=px.bar(dfm_bottom_r, x='역명', y='이용객수',
color='역명',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 역별 하차 이용객수 (상위30)
fig=px.bar(dfm_top_s, x='역명', y='이용객수',
color='역명',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)

In [12]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# 역별 하차 이용객수 (하위30)
fig=px.bar(dfm_bottom_s, x='역명', y='이용객수',
color='역명',
color_continuous_scale=px.colors.diverging.Spectral)
fig.update_layout(width=900)